# ODM2 Example 2: Load Data into ODM2 from an Excel Template File

This example shows how to load data from an ODM2 Specimen Excel Template into an ODM2 SQLite database instance using the ODM2 YODA Tools library and the ODM2 Python application programming interface (API). This example uses SQLite for the database because it doesn't require a server. However, the process for creating ODM2 databases using other relational database management systems is very similar. The ODM2 Python API and YODA Tools demonstrated here can be used with ODM2 databases implemented in:
* Microsoft SQL Server
* MySQL
* PostgresSQL
* SQLite

Details of the Specimen Excel Template (and others) for ODM2 can be found at: https://github.com/ODM2/YODA-File/tree/master/excel_templates. These Excel Templates were designed for investigators to enter their observations and metadata for parsing into an ODM2 database. We designed these templates under the premise that most scientists have and use Excel and can load thier data into the teamplate.

Details of the YODA Tools libraries can be found at: https://github.com/ODM2/YODA-Tools. YODA Tools is a code base for working with ODM2 related files, loading them into ODM2 instances, and exporting from ODM2 to files.

Details of the ODM2 Python API can be found at: https://github.com/ODM2/ODM2PythonAPI. The ODM2 Python API is an application programming interface for ODM2 databases that is cross platform and cross database compatible.

First, load the required libraries.

In [8]:
import os
import sqlite3
import sys

# sys.path.insert(0, "code")

#from yodatools.converter.Inputs.ExcelInput import ExcelInput
from yodatools.converter.Inputs import ExcelInput

from yodatools.converter.Outputs.dbOutput import dbOutput
from yodatools.converter.Outputs.yamlOutput import yamlOutput

from IPython.core.display import display, HTML
print("Done loading libraries!")

Done loading libraries!


In [9]:
# JUST FOR CHECKING PACKAGE VERSION (USE DURING DEV ONLY)
import sqlalchemy
sqlalchemy.__version__

'1.1.7'

### Create a New ODM2 Database to Load Data Into

Create a blank ODM2 database into which we can load data. This is the same process from Example 1.

In [10]:
# temporary hard-wiring; ultimately, need to be able to extract 
# the base directory path automatically
dpth = "/home/mayorga/Desktop/CZOData-BiGCZ-NOW/2013_NSF_BiGCZ_SSI/ProjectWork/ProjectMeetings/2017_11_UserWorkshop/wshp2017_tutorial_content"

dbname_sqlite = "ODM2_Example2.sqlite"

# PROBABLY BEST IF THE PATH TO THE SQLITE FILE IS DEFINED ONCE, ABOVE, AS A VARIABLE, 
# THEN REUSED THROUGHOUT THE NOTEBOOK

sqlite_pth = os.path.join(dpth, "data", dbname_sqlite)

In [12]:
# First check to see if the ODM2 SQLite file already exists from previous runs of this example. If so, delete it.
if os.path.isfile(sqlite_pth):
    os.remove(sqlite_pth)

# Create a new SQLite database and get a cursor
conn = sqlite3.connect(sqlite_pth)
c = conn.cursor()

# Open the DDL SQL file for ODM2
with open('../code/ODM2_for_SQLite.sql', 'r') as sqlf:
    sqlString = sqlf.read()

# Execute the DDL SQL script on the blank SQLite database
c.executescript(sqlString)

# Close the connection to the database
conn.close()

# Load the controlled vocabularies
# note the need to have 3 slashes!
dbconn_str = "sqlite:///" + sqlite_pth
%run ../code/cvload.py $dbconn_str
# %run code/cvload.py sqlite:///data/ODM2_Example2.sqlite

print("Done creating ODM2 database!")

Loading CVs using connection string: sqlite:////home/mayorga/Desktop/CZOData-BiGCZ-NOW/2013_NSF_BiGCZ_SSI/ProjectWork/ProjectMeetings/2017_11_UserWorkshop/wshp2017_tutorial_content/data/ODM2_Example2.sqlite
[===========================] 100% CV_Terms LoadedCVMediumType'> LoadedLoadeddedaded
CV Load has completed
Done creating ODM2 database!


### Read the Excel Template Data File

The HydroShare resource containing this notebook also contains an ODM2 Excel Template file in the "data" directory that contains a bunch of data derived from water quality samples collected at monitoring sites that are part of our iUTAH Gradients Along Mountain to Urban Transitions (GAMUT) monitoring network. This code opens the Excel template file and parses it using YODA Tools and the ODM2 Python API. Once the Excel Template file has been parsed, all of the data in the Excel file are available in the ODM2 Python API objects and can be accessed via code.

**NOTE:  This Excel template file contains a fairly large number of samples, so it takes a few seconds to parse.**

In [13]:
dbname = 'YODA_iUTAH_Specimen_Example_small.xlsx'

excel = ExcelInput(os.path.join(dpth, "data", dbname))
excel.parse()
session = excel.sendODM2Session()
print("Done parsing Excel file!")

# Provide a link to download the Excel file that was just loaded into the API
print("\nYou can download the Excel file that was just loaded using the following link:\n")

# display(HTML('<a href=%s target="_blank">%s<a>' % ('data/%s' % dbname, dbname)))
display(HTML('<a href=%s target="_blank">%s<a>' % ('../data/%s' % dbname, dbname)))

TypeError: __init__() takes exactly 1 argument (2 given)

### OK, IT LOOKS LIKE THIS VERSION OF YODA TOOLS DOESN'T HAVE THE `ExcelInput` functionality that's needed

### Get Data from the Current API Session

At this point, the data from the Excel Template file now exist in memory in the ODM2 Python API objects. We can do several things with the data now, including manipulating it or using it for visualization or analysis. We could write the data out to an operational ODM2 database, or we can write the data out to a YODA file.

The following is a quick example of a simple query to the current session where the data are now held in memory. Use **session.query(ODM2Entity).all( )** where "session" refers to the current API session and "ODM2Entity" is the name of the ODM2 entity whose objects you want to return. In this example, we'll query all of the **Methods** from the session. If you want to return other objects, you could replace "Methods" with "Variables," "People," "ProcessingLevels," etc. and then change the attributes that get printed out to the console.

In [ ]:
# Get all of the Methods that were loaded from the Excel file

# EMILIO: CHANGE THIS SO IT'S NOT USING "import *"
from odm2api.ODM2.models import *

methods = session.query(Methods).all()
# Print some of the attributes of the methods
for x in methods:
    print("MethodCode: " + x.MethodCode + ", MethodName: " 
          + x.MethodName + ", MethodTypeCV: " + x.MethodTypeCV)

### Write the Data to the ODM2 Database

Now that the Excel template file has been parsed, all of the data exist in the API objects. The following code actually writes the data to the empty ODM2 SQLite database created above. Although I'm using SQLite for this example to avoid needing a separate database server, this functionality will also work with Microsoft SQL Server, MySQL, and PostgresSQL. 

**NOTE: This Excel template file contains a fairly large number of samples, so it will take a bit to write it all to the SQLite database.**

You can download the ODM2 SQLite file that has been populated with the data from the Excel Template file using the link that is printed when you run this code. **The third notebook example (ODM2_Example2.ipynb) shows how to connect to a populated ODM2 database and retrieve data using the ODM2 Python API.**

In [ ]:
# Write the data to the database, using the connection string to the ODM2 database created above
do = dbOutput()
do.save(session, dbconn_str)

# Provide a link to the ODM2 SQLite file that the data were written to
print("\nYou can download the ODM2 SQLite database populated with data using the following link:\n")

display(HTML('<a href=%s target="_blank">%s<a>' % ('data/%s' % dbname_sqlite, dbname_sqlite)))

### Write the Data to a YODA File
The data contained in the API objects can also be written out to a YAML Observations Data Archive (YODA) file.  After running the following code, you can download and examine the YAML file using the link that is printed.

In [ ]:
# Write the output to the YODA file
yo = yamlOutput()
yo.save(session, 'data/ODM2_Example2.yaml')

# Provide a link to download the YODA file created
print("\nYou can download the populated YODA file using the following link:\n")
dbname = 'ODM2_Example2.yaml'
display(HTML('<a href=%s target="_blank">%s<a>' % ('data/%s' % dbname, dbname)))